In [3]:
import pandas as pd

In [105]:
from rectpack import SORT_AREA, PackingMode, PackingBin, newPacker

from rectpack import (MaxRectsBl, MaxRectsBssf, MaxRectsBaf, MaxRectsBlsf,
SkylineBl, SkylineBlWm, SkylineMwf, SkylineMwfl, SkylineMwfWm, SkylineMwflWm,
GuillotineBssfSas, GuillotineBssfSlas, GuillotineBssfLlas, GuillotineBssfMaxas,
GuillotineBssfMinas, GuillotineBlsfSas, GuillotineBlsfLas, GuillotineBlsfSlas, GuillotineBlsfLlas, 
                     GuillotineBlsfMaxas, GuillotineBlsfMinas, GuillotineBafSas, GuillotineBafLas,
                     GuillotineBafSlas, GuillotineBafLlas, GuillotineBafMaxas, GuillotineBafMinas)
from ShakerBFDProMax import ShakerBFDProMax
from converterFromRectPack import convert
from Base.bpReadWrite import ReadWrite
from os import path, listdir, mkdir
input_folder = './our_tests'
output_folder = './results'
cases = []
for f in listdir(input_folder):
    cases.append(f)
if not path.exists(output_folder):
    mkdir(output_folder)


In [106]:
def solve_shaker(test_state, packing = GuillotineBafMinas):
    packer = ShakerBFDProMax(pack_algo=packing, rotation = False)
    for box in test_state.boxes_open:
        packer.add_rect(box.w, box.h)
    # Add the bins where the rectangles will be placed
    packer.add_bin(*test_state.bin_size, float("inf"))
    packer.pack()
    solution = convert(packer, test_case.bin_size)
    return solution

In [107]:
packings = (MaxRectsBl, MaxRectsBssf, MaxRectsBaf, MaxRectsBlsf,
SkylineBl, SkylineBlWm, SkylineMwf, SkylineMwfl, SkylineMwfWm, SkylineMwflWm,
GuillotineBssfSas, GuillotineBssfSlas, GuillotineBssfLlas, GuillotineBssfMaxas,
GuillotineBssfMinas, GuillotineBlsfSas, GuillotineBlsfLas, GuillotineBlsfSlas, GuillotineBlsfLlas, 
                     GuillotineBlsfMaxas, GuillotineBlsfMinas, GuillotineBafSas, GuillotineBafLas,
                     GuillotineBafSlas, GuillotineBafLlas, GuillotineBafMaxas, GuillotineBafMinas)

In [108]:
df = pd.DataFrame(columns=['algo', 'packing', 'test_case', 'boxes_count'])

In [109]:
from tqdm import tqdm
for case in cases:
    test_case = ReadWrite.read_state(path=path.join(input_folder, case))
    for packing in tqdm(packings):
        solution = solve_shaker(test_case, packing)
        df = pd.concat([pd.DataFrame([['shaker BFD pro', packing, case, len(solution.bins)]], columns=df.columns), df], ignore_index=True)
    tqdm._instances.clear()
    #test_case = ReadWrite.read_state(path=path.join(input_folder, case))
   # if not solution.is_valid(test_case):
       # print(f'alarm, wrong solution!!! on {case}')
        #break
    

100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [00:10<00:00,  2.46it/s]


In [138]:
from Base.bp2DSimpleHeuristics import first_fit, max_rest, single_type_heuristic

In [139]:
simple_heuristics = [first_fit, max_rest]

In [141]:
from tqdm import tqdm
for case in cases:
    test_case = ReadWrite.read_state(path=path.join(input_folder, case))
    for heur in simple_heuristics:
        single_type_heuristic(test_case, heur)
        df = pd.concat([pd.DataFrame([[str(heur), 'orgs bottom-left', case, len(test_case.bins)]], columns=df.columns), df], ignore_index=True)
    tqdm._instances.clear()
    #test_case = ReadWrite.read_state(path=path.join(input_folder, case))
   # if not solution.is_valid(test_case):
       # print(f'alarm, wrong solution!!! on {case}')
        #break

In [142]:
df['boxes_count'] = df['boxes_count'].astype(int)
df['packing'] = df['packing'].astype(str)

In [143]:
df.to_csv("algos_comparison.csv", index=False)

In [113]:
df.groupby('test_case').apply(lambda g:g.sort_values("boxes_count", ascending=False))

algo  \
test_case                          
extremes_100 540  shaker BFD pro   
             554  shaker BFD pro   
             565  shaker BFD pro   
             564  shaker BFD pro   
             563  shaker BFD pro   
...                          ...   
uniform_500  18   shaker BFD pro   
             20   shaker BFD pro   
             4    shaker BFD pro   
             23   shaker BFD pro   
             24   shaker BFD pro   

                                                           packing  \
test_case                                                            
extremes_100 540  <class 'rectpack.guillotine.GuillotineBafMinas'>   
             554  <class 'rectpack.guillotine.GuillotineBssfLlas'>   
             565          <class 'rectpack.maxrects.MaxRectsBssf'>   
             564           <class 'rectpack.maxrects.MaxRectsBaf'>   
             563          <class 'rectpack.maxrects.MaxRectsBlsf'>   
...                                                            ...   
uniform_500  18            <class 'rectpack.skyline.SkylineMwfWm'>   
             20              <class 'rectpack.skyline.SkylineMwf'>   
             4      <class 'rectpack.guillotine.GuillotineBafLas'>   
             23           <class 'rectpack.maxrects.MaxRectsBlsf'>   
             24            <class 'rectpack.maxrects.MaxRectsBaf'>   

                     test_case  boxes_count  
test_case                                    
extremes_100 540  extremes_100           51  
             554  extremes_100           51  
             565  extremes_100           51  
             564  extremes_100           51  
             563  extremes_100           51  
...                        ...          ...  
uniform_500  18    uniform_500          146  
             20    uniform_500          146  
             4     uniform_500          146  
             23    uniform_500          146  
             24    uniform_500          146  

[567 rows x 4 columns]

In [144]:
df.groupby('test_case')['boxes_count'].min()

test_case
extremes_100           51
extremes_1000         504
extremes_50            26
extremes_500          252
known_1000_bins_66      7
known_100_bins_6        7
known_500_bins_33       7
known_50_bins_3         7
medium_500            108
meduim_100             33
meduim_1000           354
meduim_50              17
meduim_500            172
small_100               4
small_1000             36
small_50                2
small_500              18
uniform_100            32
uniform_1000          295
uniform_50             16
uniform_500           146
Name: boxes_count, dtype: int32

In [36]:
df[df['test_case'] == "uniform_500"]['boxes_count'].max()

158

In [80]:
df

,algo,packing,test_case,boxes_count
0,shaker BFD pro,<class 'rectpack.guillotine.GuillotineBafMinas'>,uniform_500,148
1,shaker BFD pro,<class 'rectpack.guillotine.GuillotineBafMaxas'>,uniform_500,151
2,shaker BFD pro,<class 'rectpack.guillotine.GuillotineBafLlas'>,uniform_500,151
3,shaker BFD pro,<class 'rectpack.guillotine.GuillotineBafSlas'>,uniform_500,149
4,shaker BFD pro,<class 'rectpack.guillotine.GuillotineBafLas'>,uniform_500,146
...,...,...,...,...
716,shaker BFD pro,<class 'rectpack.maxrects.MaxRectsBlsf'>,extremes_100,51
717,shaker BFD pro,<class 'rectpack.maxrects.MaxRectsBaf'>,extremes_100,51
718,shaker BFD pro,<class 'rectpack.maxrects.MaxRectsBssf'>,extremes_100,51
719,shaker BFD pro,<class 'rectpack.maxrects.MaxRectsBl'>,extremes_100,51


In [127]:
best_packers = [GuillotineBafSlas, GuillotineBlsfSlas, GuillotineBafMinas, GuillotineBafLas, MaxRectsBssf, MaxRectsBlsf, MaxRectsBaf]

In [151]:
df[df['test_case'] == 'uniform_1000'].sort_values('boxes_count')

,algo,packing,test_case,boxes_count
100,shaker BFD pro,<class 'rectpack.guillotine.GuillotineBafLas'>,uniform_1000,295
120,shaker BFD pro,<class 'rectpack.maxrects.MaxRectsBaf'>,uniform_1000,296
121,shaker BFD pro,<class 'rectpack.maxrects.MaxRectsBssf'>,uniform_1000,296
119,shaker BFD pro,<class 'rectpack.maxrects.MaxRectsBlsf'>,uniform_1000,297
107,shaker BFD pro,<class 'rectpack.guillotine.GuillotineBlsfSas'>,uniform_1000,297
122,shaker BFD pro,<class 'rectpack.maxrects.MaxRectsBl'>,uniform_1000,298
105,shaker BFD pro,<class 'rectpack.guillotine.GuillotineBlsfSlas'>,uniform_1000,298
116,shaker BFD pro,<class 'rectpack.skyline.SkylineMwf'>,uniform_1000,298
114,shaker BFD pro,<class 'rectpack.skyline.SkylineMwfWm'>,uniform_1000,298
101,shaker BFD pro,<class 'rectpack.guillotine.GuillotineBafSas'>,uniform_1000,298


In [120]:
grouped = df.groupby('test_case')

In [121]:
for_ranks = grouped.apply(lambda x: x.reset_index(drop = True)).reset_index(drop = True)
for_ranks['rank'] = grouped['boxes_count'].rank(method='min',  ascending=True)
for_ranks

,algo,packing,test_case,boxes_count,rank
0,shaker BFD pro,<class 'rectpack.guillotine.GuillotineBafMinas'>,extremes_100,51,10.0
1,shaker BFD pro,<class 'rectpack.guillotine.GuillotineBafMaxas'>,extremes_100,51,18.0
2,shaker BFD pro,<class 'rectpack.guillotine.GuillotineBafLlas'>,extremes_100,51,18.0
3,shaker BFD pro,<class 'rectpack.guillotine.GuillotineBafSlas'>,extremes_100,51,12.0
4,shaker BFD pro,<class 'rectpack.guillotine.GuillotineBafLas'>,extremes_100,51,1.0
...,...,...,...,...,...
562,shaker BFD pro,<class 'rectpack.skyline.SkylineBl'>,uniform_500,158,1.0
563,shaker BFD pro,<class 'rectpack.maxrects.MaxRectsBlsf'>,uniform_500,146,1.0
564,shaker BFD pro,<class 'rectpack.maxrects.MaxRectsBaf'>,uniform_500,146,1.0
565,shaker BFD pro,<class 'rectpack.maxrects.MaxRectsBssf'>,uniform_500,147,1.0


In [102]:
df.groupby(['test_case', 'packing'])['boxes_count'].count()

test_case     packing                                         
extremes_100  <class 'rectpack.guillotine.GuillotineBafLas'>      6
              <class 'rectpack.guillotine.GuillotineBafLlas'>     6
              <class 'rectpack.guillotine.GuillotineBafMaxas'>    6
              <class 'rectpack.guillotine.GuillotineBafMinas'>    6
              <class 'rectpack.guillotine.GuillotineBafSas'>      6
                                                                 ..
uniform_500   <class 'rectpack.skyline.SkylineBlWm'>              1
              <class 'rectpack.skyline.SkylineMwf'>               1
              <class 'rectpack.skyline.SkylineMwfWm'>             1
              <class 'rectpack.skyline.SkylineMwfl'>              1
              <class 'rectpack.skyline.SkylineMwflWm'>            1
Name: boxes_count, Length: 568, dtype: int64

In [46]:
grouped['boxes_count'].rank(method='min')

0      10.0
1      18.0
2      18.0
3      12.0
4       1.0
       ... 
716     1.0
717     1.0
718     1.0
719     1.0
720     1.0
Name: boxes_count, Length: 721, dtype: float64